In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
extracted = pd.read_excel(r'data/extracted_data.xlsx')
correction = pd.read_excel(r'data/agent_correction_requests.xlsx')

data = extracted.copy()
corrected = correction.copy()

In [5]:
class Data:
    def __init__(self, df1, df2):
        self.df1 = df1
        self.df2 = df2
    def clean_reference(self) -> pd.DataFrame:
        self.df1['Account Reference'] = self.df1['Account Reference'].str.replace('\'','')
        return self.df1

    def split_text(self) -> pd.DataFrame:
        notes = self.df1[self.df1['Notes'].str.len() > 100]
        notes.loc[:, 'Notes'] = notes.Notes.str[100:]
        self.df1 = pd.concat([self.df1, notes])
        self.df1.Notes = self.df1.Notes.str[0:101]
        self.df1 = self.df1.sort_index().reset_index(drop=True)
        return self.df1

    def clean_data(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        corr = self.df2.copy().astype(str)
        corr['Date'] = corr['Date'] + " " + corr['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(corr['Date'])) & (cleaned['Account Reference'].isin(corr['Account number']))]

        for i in range(len(cleaned)):
            cleaned[corr.iloc[i,4]].iloc[i] = corr.iloc[i,5]
        self.df1.update(cleaned)
        self.df1.drop_duplicates(inplace=True, ignore_index=True)
        return self.df1

    def ib_arrangement(self, cleaned: pd.DataFrame):
        cleaned: pd.DataFrame = cleaned[(cleaned['PTP Date'] != "0000-00-00 00:00:00") & (cleaned['PTP Amount'] > 0) & (cleaned['Disposition Desc'] == 'PROMISE TO PAY')]

        cleaned: pd.DataFrame = pd.DataFrame({
            'Account Reference': cleaned['Account Reference'],
            'Instalment Date': cleaned['PTP Date'],
            'Arrangement Amount': cleaned['PTP Amount'],
            'Instalment Paid Value': "0",
            'Instalment Paid Date': "",
            'Settlement Program': "PTP",
            'With Post Dated Check': "N"
        })
        cleaned['Arrangement Amount'] = cleaned['Arrangement Amount'].apply(lambda x: "{:.2f}".format(x))
        cleaned['Instalment Date'] = pd.to_datetime(cleaned['Instalment Date']).dt.strftime('%m/%d/%Y')
        cleaned.reset_index(drop=True, inplace=True)
        return cleaned


    def correction_status(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        self.df2 = self.df2.astype(str)
        self.df2['Date'] = self.df2['Date'] + " " + self.df2['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(self.df2['Date'])) & (cleaned['Account Reference'].isin(self.df2['Account number']))]
        self.df2['Status'] = np.where(self.df2['Account number'].isin(cleaned['Account Reference']), "Corrected", "Not Found")
        return self.df2

    def save_data(self, main_1: pd.DataFrame, main_2: pd.DataFrame, file_2: pd.DataFrame):
        main_file: str = input('Main File Name: ')
        second_file: str = input('Second File Name: ')
        yesterday = (dt.date.today() - dt.timedelta(1)).strftime('%m%d%Y')

        with pd.ExcelWriter(f'{main_file}_{yesterday}.xlsx') as writer:
            main_1.to_excel(writer, sheet_name='All Data' ,index=False)
            main_2.to_excel(writer, sheet_name='IB Arrangement' ,index=False)

        with pd.ExcelWriter(f'{second_file}_{yesterday}.xlsx') as writer:
            file_2.to_excel(writer, sheet_name='IB Arrangement' ,index=False)


In [6]:
if __name__ == '__main__':
    output = Data(data, corrected)
    output.clean_reference()
    output.split_text()
    output.save_data(output.clean_data(), output.ib_arrangement(output.clean_data()), output.correction_status())

C:\Users\Matthew\AppData\Local\Temp\ipykernel_26244\2684896032.py:24: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  cleaned[corr.iloc[i,4]].iloc[i] = corr.iloc[i,5]
C:\Users\Matthew\AppData\Local\Temp\ipykernel_26244\2684896032.py:24: Future

In [ ]:
output =